In [1]:
# %pip install torch torchvision torchaudio


In [2]:
from pipeline.build_dataset import make_state_frame
import yaml

cfg = yaml.safe_load(open("config/data_config.yaml"))
dataset = make_state_frame("AAPL", cfg)
# print(dataset.tail())
# dataset.tail().isna().sum()
# raw = fetch_ohlcv("AAPL", cfg["start_date"], cfg["end_date"])
# print(raw.index[-5:], dataset.index[-5:])


[*********************100%***********************]  1 of 1 completed


## Assembler

In [3]:
from pipeline.state_assembler import StateAssembler

# removing empty items
dataset = dataset.dropna().copy()

# # Assemble rolling-window states
# assembler = StateAssembler(
#     feature_cols=[c for c in dataset.columns if c != "price"],
#     window_size=30
# )
# state_df = assembler.assemble(dataset)
# # print(state_df.shape)
# # print(state_df.tail())
# # 3. Align prices by using the *same index* as state_df
# prices = dataset.loc[state_df.index, "price"]

# print("state_df shape:", state_df.shape)
# print("prices shape:", prices.shape)
from envs.buy_env import BuyEnv
# from state_assembler import StateAssembler
import numpy as np

# Assuming you’ve already built `dataset` with your indicators + price
feature_cols = ["return_1d", "rsi14", "macd_diff", "bb_b",
                "atr14", "roc10", "obv", "mfi14", "willr14"]

assembler = StateAssembler(feature_cols=feature_cols, window_size=30)
state_df = assembler.assemble(dataset)

# Align prices index with state_df
prices = dataset.loc[state_df.index, "price"]

env = BuyEnv(state_df, prices, horizon=5, transaction_cost=0.001)

obs = env.reset()
print("Initial obs shape:", obs.shape, "dtype:", obs.dtype)

for step in range(10_000):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    if step % 1000 == 0:
        print(f"step={step}, reward={reward}, done={done}")
    if done:
        obs = env.reset()

print("Random roll-out finished without crash.")


Initial obs shape: (270,) dtype: float32
step=0, reward=0.01565893366932869, done=True
step=1000, reward=0.0, done=False
step=2000, reward=0.0, done=False
step=3000, reward=-0.005347793456166983, done=True
step=4000, reward=0.0, done=False
step=5000, reward=0.0, done=False
step=6000, reward=0.0, done=False
step=7000, reward=0.01565893366932869, done=True
step=8000, reward=0.0, done=False
step=9000, reward=0.0, done=False
Random roll-out finished without crash.


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


## Buy Env

In [4]:
from envs.buy_env import BuyEnv
import numpy as np


# # 3) Align prices

# dataset = dataset.dropna().copy()
# prices = dataset["price"].copy()
# states = dataset.drop(columns=["price"]).copy()

# price_aligned = dataset.loc[state_df.index, "price"]

# print("state_df shape:", state_df.shape)
# print("prices shape:", prices.shape)

# print("NaNs in state_df:", np.isnan(state_df.values).sum())
# print("NaNs in prices:", np.isnan(prices.values).sum())


# 4) Create environment
# env = BuyEnv(state_df, price_aligned, horizon=5, transaction_cost=0.001)
env = BuyEnv(state_df, prices, horizon=5, transaction_cost=0.001)

# 5) Smoke test
obs = env.reset()
print("Initial state shape:", obs.shape)

for step in range(5):
    action = env.action_space.sample()  # random action
    next_obs, reward, done, info = env.step(action)
    print(f"step={step}, action={action}, reward={reward:.4f}, done={done}, info={info}")
    if done:
        break

# print("Initial obs shape:", obs.shape)

# from stable_baselines3 import DQN

# model = DQN("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=5_000)

# obs = env.reset()
# action = model.predict(obs, deterministic=True)[0]
# print("Greedy action:", action)



Initial state shape: (270,)
step=0, action=0, reward=0.0000, done=False, info={'t': 1, 'price': 47.02927780151367}
step=1, action=1, reward=0.0085, done=True, info={'t': 2, 'price': 47.76933670043945}


In [5]:
# from stable_baselines3 import DQN
# from stable_baselines3.common.vec_env import DummyVecEnv

# # # Rebuild env exactly as above
# # feature_cols = ["return_1d", "rsi14", "macd_diff", "bb_b",
# #                 "atr14", "roc10", "obv", "mfi14", "willr14"]
# # assembler = StateAssembler(feature_cols=feature_cols, window_size=30)
# # state_df = assembler.assemble(dataset)
# # prices = dataset.loc[state_df.index, "price"]

# # env = BuyEnv(state_df, prices, horizon=5, transaction_cost=0.001)

# # Wrap explicitly in DummyVecEnv (SB3 does this internally, but we make it explicit)
# vec_env = DummyVecEnv([lambda: env])

# model = DQN(
#     "MlpPolicy",
#     vec_env,
#     verbose=1,
#     learning_rate=1e-4,
#     buffer_size=5_000,
#     batch_size=32,
#     train_freq=1,
#     gradient_steps=1,
#     exploration_fraction=0.1,
#     exploration_final_eps=0.01,
#     target_update_interval=500,
# )

# model.learn(total_timesteps=5_000)


In [ ]:
# scripts/train_buy_ddqn.py

import os
import sys
import random
import numpy as np
import torch

from agents.buy_agent_ddqn import BuyAgentTrainer

def set_seeds(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


if __name__ == "__main__":
    # Just to be safe / reproducible
    set_seeds(42)

    # Make sure we run from project root
    os.chdir(os.path.dirname(os.path.dirname(__file__)))

    # 1) create trainer for one ticker (e.g. AAPL)
    trainer = BuyAgentTrainer(
        cfg_path="config/data_config.yaml",
        ticker="AAPL",
        window_size=30,
        horizon=5,
        transaction_cost=0.001,
    )

    # 2) quick training run (PoC)
    #    start small (e.g. 50 episodes) to make sure everything works
    rewards = trainer.train(
        n_episodes=50,
        max_steps_per_episode=None,  # let env end naturally
        warmup_steps=500,
    )

    print("Last 10 episode rewards:", rewards[-10:])

    # 3) run greedy policy once to see how it behaves
    greedy_policy = trainer.make_greedy_policy()
    total_reward, steps = greedy_policy(trainer.env)
    print(f"Greedy run: total_reward={total_reward:.4f}, steps={steps}")


NameError: name '__file__' is not defined

## Training agent

In [ ]:
# from agents.buy_agent_ddqn import BuyAgentDDQN

# # Reuse env from previous block
# buy_agent = BuyAgentDDQN(env, learning_rate=1e-4, gamma=0.99)

# # Short training run just to validate everything works
# buy_agent.train(timesteps=10_000)

# # Save model
# buy_agent.save("results/models/buy_agent_aapl_ddqn")

# # Test one greedy decision
# obs = env.reset()
# action = buy_agent.act(obs)
# print("Greedy action at start:", action)

